# Import

In [ ]:
# %% import stuff
from re import S
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from scipy.interpolate import griddata


In [ ]:
import matplotlib as mpl
mpl.rcParams['axes.grid'] = False

# Make illustration for sampling strategy

## 2D

In [ ]:
b_arr = np.arange(-0.25, 0.5001, 0.05)
a_arr = np.full(len(b_arr), 0.5)
b_arr2 = np.full(len(b_arr), -0.25)

a_arr, b_arr = np.append(a_arr, b_arr), np.append(b_arr, b_arr2)
print('a_arr, b_arr:')
print(a_arr, b_arr)

%matplotlib qt
x = np.vstack((a_arr, np.zeros(len(b_arr))))
y = np.vstack((b_arr, np.zeros(len(b_arr))))
print('x:')
print(x)
plt.plot(x, y)
plt.scatter(a_arr,b_arr)
plt.gca().set_aspect('equal')
plt.xlabel('a')
plt.ylabel('b')
plt.grid()
# plt.savefig('ab_plot.png')


## 3D

In [ ]:
ab_var_arr = np.arange(-0.25, 0.5001, 0.05).reshape(-1, 1)
c_var_arr = np.arange(0, 0.5001, 0.05)
n_ab, n_c = len(ab_var_arr), len(c_var_arr)

# plane 1: a constant
arrs_a_const = np.empty((3, n_ab, n_c))
arrs_a_const[0] = 0.5
arrs_a_const[1] = np.tile(ab_var_arr, reps=(1,n_c))
arrs_a_const[2] = np.tile(c_var_arr, reps=(n_ab, 1))

# plane 2: b constant
arrs_b_const = np.empty((3, n_ab, n_c))
arrs_b_const[0] = np.tile(ab_var_arr, reps=(1,n_c))
arrs_b_const[1] = -0.25
arrs_b_const[2] = np.tile(c_var_arr, reps=(n_ab, 1))

# plane 3: c constant
arrs_c_const = np.empty((3, n_ab, n_ab))
arrs_c_const[0] = np.tile(ab_var_arr, reps=(1,n_ab))
arrs_c_const[1] = np.tile(ab_var_arr, reps=(1,n_ab)).T
arrs_c_const[2] = 0.5

abc_arr = np.concatenate((arrs_a_const.reshape(3, -1),
                    arrs_b_const.reshape(3, -1),
                    arrs_c_const.reshape(3, -1)), -1)
abc_arr = abc_arr[:, abc_arr[0] > abc_arr[1]]
print(abc_arr.shape)


In [ ]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

steps = np.linspace(0, 1, 101)
for xx, yy, zz in abc_arr.T:
    ax.plot(xx*steps, yy*steps, zz*steps, c='tab:orange')
ax.scatter(*abc_arr)
ax.scatter([0], [0], [0], c='black')

# ax.set_aspect('equal')
ax.set_xlabel('$G_{xx}$')
ax.set_ylabel('$G_{yy}$')
ax.set_zlabel('$G_{xy}$')
ax.grid()
plt.show()
# plt.savefig('abc_plot.png')

# Plots

In [ ]:
diameter = 0.9

In [ ]:
# %% open dataframe from file
with open(f'data\\coarseMesh\\dataframe_coarseMesh_diameter_{diameter}.pkl', 'rb') as f:
    df = pickle.load(f)


In [ ]:

# %% extract a, b, c
df['a'] = [elem[0][0]-1 for elem in df['F']]
df['b'] = [elem[1][1]-1 for elem in df['F']]
df['c'] = [elem[1][0] for elem in df['F']]
df


In [ ]:
plt.hist(df['c'], bins=50);

In [ ]:
# find all relevant data

# bools = np.isclose(df['c'], 0.2, atol=0.02, rtol=0)
bools = df['c'] == 0.0

a_arr, b_arr, D_arr, W_arr = df[bools][['a', 'b', 'D', 'W']].values.T
bf_points = df[df['bifurc']*(bools)][['a', 'b']]

a_lims = b_lims = [-0.25, 0.5]
print(a_lims, b_lims)

symm_type_arr = df['symm_type'][df['c'] == 0.0]

In [ ]:
# %% Find all bifurcation points for c=[some specific value] and plot them
%matplotlib qt

plt.figure(figsize=(8,8))
plt.scatter(np.append(b_arr, a_arr), np.append(a_arr, b_arr), s=1)
plt.scatter(bf_points['a'], bf_points['b'], s=2, c='tab:orange')
plt.scatter(bf_points['b'], bf_points['a'], s=2, c='tab:orange')
plt.gca().set_aspect('equal')
plt.xlabel('a')
plt.ylabel('b')
plt.xlim([-0.25, 0.5])
plt.ylim([-0.25, 0.5])
plt.grid()
# plt.savefig(f'bifurcation_diameter_{diameter}.png')



## Plot strain energy density

In [ ]:
W_min, W_max = np.abs(np.min(W_arr)), np.abs(np.max(W_arr))
W_max = max(W_min, W_max)
W_min = - W_max

In [ ]:
# %% Plot energy

b_sample, a_sample = np.ix_(np.linspace(*a_lims, 100),
                            np.linspace(*b_lims, 50))
grid_z0 = griddata((a_arr, b_arr), W_arr, (a_sample, b_sample))

%matplotlib qt
plt.figure(figsize=(8,8))
cf = plt.contourf(a_sample.flatten(), b_sample.flatten(), grid_z0, 50)
    # cmap='coolwarm', vmin=W_min, vmax=W_max)

plt.contour(a_sample.flatten(), b_sample.flatten(), grid_z0, 100,
    c='black')
plt.scatter(a_arr, b_arr, s=1)
plt.scatter(bf_points['a'], bf_points['b'], s=2, c='tab:orange')
plt.gca().set_aspect('equal')
plt.xlabel('$G_{xx}$')
plt.ylabel('$G_{yy}$')
plt.grid()
cbar = plt.colorbar(cf)
cbar.ax.set_ylabel('$\mathfrak{W}$')
# cbar.set_ticks(cbar.get_ticks())
# cbar.set_ticklabels(10**cbar.get_ticks())


## Plot one component of D

In [ ]:
# %% import stuff
from re import S
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from scipy.interpolate import griddata

In [ ]:
diameter = 0.9

In [ ]:
# %% open dataframe from file
with open(f'data\\coarseMesh\\dataframe_coarseMesh_diameter_{diameter}.pkl', 'rb') as f:
    df = pickle.load(f)


In [ ]:

# %% extract a, b, c
df['a'] = [elem[0][0]-1 for elem in df['F']]
df['b'] = [elem[1][1]-1 for elem in df['F']]
df['c'] = [elem[1][0] for elem in df['F']]
df


In [ ]:
# find all relevant data

# bools = np.isclose(df['c'], 0.2, atol=0.02, rtol=0)
bools = df['c'] == 0.0

a_arr, b_arr, D_arr, W_arr = df[bools][['a', 'b', 'D', 'W']].values.T
bf_points = df[df['bifurc']*(bools)][['a', 'b']]

a_lims = b_lims = [-0.25, 0.5]
print(a_lims, b_lims)

symm_type_arr = df['symm_type'][df['c'] == 0.0]

In [ ]:
import matplotlib.colors

In [ ]:
colors = [matplotlib.colormaps['tab10'](sytype) for sytype in symm_type_arr]

In [ ]:
%matplotlib qt
D1 = [elem[1,1,1,1] for elem in D_arr]
D2 = [elem[0,0,0,0] for elem in D_arr]
D_min, D_max = np.abs(np.min(D1)), np.abs(np.max(D1))
D_max = max(D_min, D_max)
D_min = - D_max

D1 = np.concatenate((D1, D2))
a_arr2, b_arr2 = np.concatenate((a_arr, b_arr)), np.concatenate((b_arr, a_arr))
colors2 = np.concatenate((colors, colors))

# %% Plot stiffness
b_sample, a_sample = np.ix_(np.linspace(*a_lims, 50),
                            np.linspace(*b_lims, 50))
grid_z0 = griddata((a_arr2, b_arr2), D1, (a_sample, b_sample), method='cubic')

plt.figure(figsize=(8,8))
cf = plt.contourf(a_sample.flatten(), b_sample.flatten(), grid_z0, 50,
                    vmin=D_min, vmax=D_max, cmap='coolwarm')
# cf = plt.contour(a_sample[:, 0], b_sample[0], grid_z0, 100)  #, c='black')
# plt.scatter(a_arr2, b_arr2, s=1,
#             # c='black',
#             c=colors2
#             )
# plt.scatter(bf_points['a'], bf_points['b'], s=2, c='tab:orange')
# plt.scatter(bf_points['b'], bf_points['a'], s=2, c='tab:orange')
plt.gca().set_aspect('equal')
plt.xlabel('$G_{xx}$')
plt.ylabel('$G_{yy}$')
cbar = plt.colorbar(cf)
cbar.ax.set_ylabel('$D_{yyyy}$')

# %%


## Plot all D components

In [ ]:
# %% Plot stiffness
b_sample, a_sample = np.ix_(np.linspace(*a_lims, 50),
                            np.linspace(*b_lims, 50))

D_arr = np.stack(D_arr)
D_arr = D_arr.reshape(-1, 4, 4)

In [ ]:
D_min, D_max = np.abs(np.min(D_arr)), np.abs(np.max(D_arr))
D_max = max(D_min, D_max)
D_min = - D_max

In [ ]:
%matplotlib qt

fig, axes = plt.subplots(4, 4, figsize=(13,10))
titles = ['xx', 'xy', 'yx', 'yy']
for i in range(4):
    for j in range(4):
        ax = axes[i, j]
        if i > j:
            ax.axis('off')
            continue
        ax = axes[i,j]
        comp = '$D_{' + titles[i] + titles[j] + '}$'
        ax.grid('both')
        ax.set_title(comp)
        ax.set_xlabel('$G_{xx}$')
        ax.set_ylabel('$G_{yy}$')

        grid_z0 = griddata((a_arr, b_arr), D_arr[:, i, j], (a_sample, b_sample), method='cubic')

        cf = ax.contourf(a_sample.flatten(), b_sample.flatten(), grid_z0,
            50, vmin=D_min, vmax=D_max, cmap='coolwarm')
        ax.set_aspect('equal')

        ax.scatter(a_arr, b_arr, s=1, c='black')
        ax.scatter(bf_points['a'], bf_points['b'], s=2, c='tab:orange')

        if 'symm_type' in df.columns:
            ax.scatter(a_arr, b_arr, s=1, c=symm_type_arr)
        # plt.scatter(bf_points['b'], bf_points['a'], s=2, c='tab:orange')

fig.tight_layout()
fig.subplots_adjust(right=0.8, wspace=0.4, hspace=0.5)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])

cbar = plt.colorbar(cf, cax = cbar_ax)
# cbar.ax.set_ylabel('$D_{xxxx}$')

# %%


# Import mesh, plot

In [ ]:
file = ... # path to .mat file with mesh info

mat = scipy.io.loadmat(file)

for key in mat:
    print(key)
    if isinstance(mat[key], np.ndarray):
        print(mat[key].shape)
    else:
        print(mat[key])



In [ ]:

# quadratic triangles, so the elements are made up of 6 nodes, with nodes 0, 1 and 2 the corner nodes.
# minus 1 because matlab indexing starts at 1 but python's at 0
edges = mat['t'][([[0, 3, 1, 4, 2, 5], [3, 1, 4, 2, 5, 0]],)].reshape(2, -1) - 1
x, y = mat['p'][:, edges]


In [ ]:
%matplotlib qt
plt.scatter(*mat['p'], s=1)
plt.plot(x, y)
plt.gca().set_aspect('equal')
plt.grid()

# Reconstruct deformed mesh

In [ ]:
U = df.iloc[19]['U'].reshape(-1, 2).T

In [ ]:
plt.scatter(*(mat['p'] + U), s=1)
plt.gca().set_aspect('equal')

# Plot deformations

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# with open('data/graphs_coarseMesh_diameter_0.9_alts3.pkl', 'rb') as f:
with open("data\coarseMesh\graphs_coarseMesh_diameter_0.9_alts7.pkl", 'rb') as f:
    data_list0 = pickle.load(f)

print(data_list0[0])
print(len(data_list0))

In [ ]:
data_list_no_shear = [graph for graph in data_list0 if graph.F[0, 0, 1] == 0]
print(len(data_list_no_shear))

In [ ]:
F = np.concatenate([graph.F for graph in data_list_no_shear], axis=0)
F.shape

In [ ]:
%matplotlib qt

n = 7
fig, axes = plt.subplots(n, n, figsize=(10, 10))
fig.patch.set_facecolor("None")

for i in range(n):
    for j in range(n):
        ax = axes[n-1-i,j]

        # make plots pretty
        ax.set_aspect('equal')
        # plt.grid()
        ax.axis('off')

        ax.margins(0)

        # find correct graph
        interval = (1.5-0.75)/(n-1)
        F_goal1 = np.array([[0.75+interval*j, 0], [0, 0.75+interval*i]])
        F_goal2 = np.array([[0.75+interval*i, 0], [0, 0.75+interval*j]])
        # print(F_goal1)
        err1 = np.mean((F - F_goal1)**2, axis=(1,2))
        ind1 = np.argmin(err1)
        err2 = np.mean((F - F_goal2)**2, axis=(1,2))
        ind2 = np.argmin(err2)

        if err1[ind1] < err2[ind2]:
            graph = data_list_no_shear[ind1]
            pos = graph.y[..., 0]
            if err1[ind1] > 0.0001:
                print('large error:')
                print(F_goal1)
                print(graph.F)
                print(err1[ind1])
        else:
            graph = data_list_no_shear[ind2]
            pos = graph.y[..., [1, 0], 0]  # swap x and y

            if err2[ind2] > 0.0001:
                print('large error:')
                print(F_goal2)
                print(graph.F)
                print(err2[ind2])

        # plot nodes
        # ax.scatter(*pos.T, label='new position', s=1, c='tab:blue')

        # filter out wraparound edges
        x, y = np.transpose(pos[graph.edge_index], axes=[2,0,1])
        bools = ((np.abs(np.diff(x, axis=0)) < 0.5)
                    & (np.abs(np.diff(y, axis=0)) < 0.5)
                ).flatten()
        # plot edges
        if np.isclose(F_goal1, np.identity(2)).all():
            edges0 = ax.plot(x[:, bools], y[:, bools], alpha=0.3, c='tab:blue')
        else:
            edges0 = ax.plot(x[:, bools], y[:, bools], alpha=0.3, c='black')

xlims = [-1.6, 1.6]
ylims = [-1.6, 1.6]

for ax in axes.flatten():
    xlims_temp = ax.get_xlim()
    xlims[0] = min(xlims[0], xlims_temp[0])
    xlims[1] = max(xlims[1], xlims_temp[1])
    ylims_temp = ax.get_ylim()
    ylims[0] = min(ylims[0], ylims_temp[0])
    ylims[1] = max(ylims[1], ylims_temp[1])

for ax in axes.flatten():
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)

plt.tight_layout()
plt.subplots_adjust(wspace=0.05, hspace=0.05)